<h2 align="center">Dashboard for Unemployment Rate</h2> 
<h4 align="center">United States (2010 - 2020)</h4>
<div align="center">
    <img src="index.jpg" style="height: 300px; width:400px;"/>
</div>

In [1]:
import dash
from dash import html
from dash import dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input,Output,State    
import plotly.graph_objs as go
import json

<h5>We want to see unemployment rate in US from 2010 to 2020 for each category and see unemployment rate during COVID-19 for each state.</h5>

In [2]:
DF = pd.read_csv('unemployment_data_us.csv')
DF = DF.dropna()
print('Shape of Unemployment Data US is ',DF.shape)

Shape of Unemployment Data US is  (123, 13)


<h4>Average rate in each category</h4>
<p>The unemployment rate in the Black category is highest with a rate of 10.692683, 
    followed by the Primary category with a rate of 9.329268, and the lowest unemployment rate in the Professional Degree category is at 3.127642</p>

In [3]:
DF.drop(columns=['Month','Year','Date']).mean()

Primary_School          9.329268
High_School             6.391057
Associates_Degree       5.338211
Professional_Degree     3.127642
White                   5.455285
Black                  10.692683
Asian                   4.679675
Hispanic                7.660163
Men                     5.848780
Women                   5.485366
dtype: float64

In [4]:
DF_State = pd.read_csv('unemployment_data_us_state.csv')
DF_State['Rate'] = DF_State['Unemployment_Rate']
DF_State=DF_State.dropna()
print('Shape of Unemployment Data US for State is ',DF_State.shape)

Shape of Unemployment Data US for State is  (155, 5)


<h4>Average rate in each state during COVID-19</h4>

In [5]:
unemploymentState= DF_State.drop(columns=['Date']).groupby('State').mean().sort_values(by='Rate',ascending=False)
unemploymentState.head()

,Unemployment_Rate,Rate
State,,
Puerto Rico,6.700000,6.700000
Alaska,5.800000,5.800000
Kentucky,5.100000,5.100000
West Virginia,4.933333,4.933333
Arizona,4.833333,4.833333


In [6]:
# Coordinates for  each country, helps in building the map
f = open('gz_2010_us_040_00_500k.json')
geojson = json.load(f)

In [7]:
app = dash.Dash(__name__,external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [8]:
app.layout = html.Div([
    # Title Dashboard
    html.H1('US Unemployment Rate (2010-2020)',  
            style = {'color':'White','backgroundColor':'DarkCyan','fontsize':40,'textAlign':'center'}),
    
    #  First div contains two main parts , the first for a line chart and the other for the big numbers
    html.Div([        
        #First Graph Lines for features and change data of year 
        html.Div([
                dcc.Graph(id='lineFig',figure={}),
                dcc.Slider(id='sliderLineFig',min=DF.Year.min(), 
                max=DF.Year.max(), value=DF.Year.min(),marks={str(i):str(i) for i in DF.Year.unique()})
        ], className='eight columns'),
        
        #Big Numbers
        html.Div([ 
            html.Div([ 
                html.H3(str(round(DF.Primary_School.mean(),3)),  style = {'color':'DarkCyan','textAlign':'center'}),
                html.H5('Primary School',style = {'color':'Silver','textAlign':'center'}),
            
                html.H3(str(round(DF.High_School.mean(),3)),style = {'color':'DarkCyan','textAlign':'center'}),
                html.H5('High School',style = {'color':'Silver','textAlign':'center'}), 
                
                html.H3(str(round(DF.Associates_Degree.mean(),3)),style = {'color':'DarkCyan','textAlign':'center'}),
                html.H5('Associates Degree',style = {'color':'Silver','textAlign':'center'}), 
                
                html.H3(str(round(DF.Professional_Degree.mean(),3)),style = {'color':'DarkCyan','textAlign':'center'}),
                html.H5('Professional Degree',style = {'color':'Silver','textAlign':'center'}), 
                
                html.H3(str(round(DF.White.mean(),3)),style = {'color':'DarkCyan','textAlign':'center'}), 
                html.H5('White',style = {'color':'Silver','textAlign':'center'})
            ],className='six columns'),
        
        
           html.Div([
               html.H3(str(round(DF.Black.mean(),3)),style =  {'color':'DarkCyan','textAlign':'center'}),
               html.H5('Black',style = {'color':'Silver','textAlign':'center'}), 
               
               html.H3(str(round(DF.Asian.mean(),3)),style =  {'color':'DarkCyan','textAlign':'center'}),
               html.H5('Asian',style = {'color':'Silver','textAlign':'center'}), 
               
               html.H3(str(round(DF.Hispanic.mean(),3)),style =  {'color':'DarkCyan','textAlign':'center'}),
               html.H5('Hispanic',style =  {'color':'Silver','textAlign':'center'}), 
               
               html.H3(str(round(DF.Men.mean(),3)),style =  {'color':'DarkCyan','textAlign':'center'}),
               html.H5('Men',style = {'color':'Silver','textAlign':'center'}), 
               
               html.H3(str(round(DF.Women.mean(),3)),style = {'color':'DarkCyan','textAlign':'center'}),  
               html.H5('Women',style =  {'color':'Silver','textAlign':'center'}) 
           ],className='six columns')
        ], className='four columns',style = {'backgroundColor':'GhostWhite'})
        
    ]),

    # Second div contains the two drop down for desigred features.
    html.Div([ 
               html.Div([  dcc.Dropdown(id='dropdown_1', options=[{'label':str(c), 
                    'value':str(c)}for c in DF.drop(columns=['Year','Month','Date']).columns], 
                     value='White')],className='three columns'),
                
               html.Div([  dcc.Dropdown(id='dropdown_2', options=[{'label':str(c), 
                    'value':str(c)}for c in DF.drop(columns=['Year','Month','Date']).columns], 
                     value='Black')], className='three columns')
            ]),

    # Third div contains two parts first for bar chart, the other for the map.
    html.Div([
        html.Div([
            dcc.Graph(id='barChart', figure={})
            ], className='seven columns'),
        
        
        html.Div([
            dcc.Graph(id='mapChart', figure={}, style={'margin-left' : '-110px'})
            ], className='two columns')
        
    ])

],style={'backgroundColor':'DarkCyan'})



@app.callback(
    Output(component_id='lineFig', component_property='figure'),
    Output(component_id='barChart', component_property='figure'),
    Output(component_id='mapChart', component_property='figure'),

    Input(component_id='sliderLineFig', component_property='value'),
    Input(component_id='dropdown_1', component_property='value'),
    Input(component_id='dropdown_2', component_property='value')


)
def update_output(value,dropdown_1,dropdown_2):
    """
    Takes the needed features names to change the graphs based on them, returens a new updated graphs.
    """
    
    Color = ['DarkTurquoise', 'CadetBlue','Gray','DarkSeaGreen','DarkSlateGray', 'Silver','SteelBlue','PaleTurquoise','DarkOliveGreen','SeaGreen']
    #Graph_1 Line
    lineFig = px.line(DF[DF['Year']==value],x='Month', 
                      y=DF.drop(columns=['Month','Year','Date']).columns,width=900, height=400,
                     title='A line showing unemployment rate for each category',labels={'value':'Unemployment Rate'}
                     ,color_discrete_map={i:j for i,j in zip( DF.drop(columns=['Year','Month','Date']).columns,Color)})
    lineFig.update_layout(plot_bgcolor="white")
    #lineFig.update_layout(title_font_color="DarkCyan")
    
    #Graph_2 Bar Chart
    df_mm = DF.groupby('Year').agg('mean').reset_index()

    fig_1 = go.Bar( x = list(df_mm['Year']), y = df_mm[dropdown_1], name = dropdown_1,
            text = df_mm[dropdown_1].round(), textposition = 'outside',texttemplate = '%{text: .2}', marker={'color':'LightGray'})
   
    fig_2 = go.Bar( x = df_mm['Year'], y = df_mm[dropdown_2], name = dropdown_2,text = df_mm[dropdown_2].round(),
            textposition = 'outside', texttemplate = '%{text: .2}',marker={'color':'CadetBlue'})

    fig = [fig_1, fig_2]
    layout = go.Layout(title = dropdown_1+' and '+dropdown_2, barmode = 'relative', template='simple_white',width=700,height=550)
    barChart = go.Figure(data = fig, layout = layout)

    
    #Graph_3 Map
    map_ = px.choropleth(DF_State, geojson=geojson, color="Rate",locations="State", featureidkey="properties.NAME",
                  scope='usa', width=600,height=600,color_continuous_scale="Viridis", title='                      2020 US Unemployment Rate')
    
    return lineFig, barChart, map_  

In [ ]:
# Initiating the server
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Oct/2021 10:32:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 10:32:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 10:32:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 10:32:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Oct/2021 10:32:34] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Oct/2021 10:32:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Oct/2021 10:32:34] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Oct/2021 10:32:38] "POST /_dash-update-component HTTP/1.1" 200 -
